In [ ]:
import scipp as sc

table = sc.data.table_xyz(1000)
da = table.bin(x=10)
da

In [ ]:
y = sc.linspace('y', 0, 100, 30, unit='m').fold('y', sizes={'x': 10, 'y': -1})
da.bin(y=y)

In [ ]:
import scipp as sc
from typing import NewType, Dict, Tuple
from scippneutron.tof import chopper_cascade, unwrap

RawData = NewType('RawData', sc.DataArray)
TofData = NewType('TofData', sc.DataArray)
FramePeriod = NewType('FramePeriod', sc.Variable)
FrameBounds = NewType('FrameBounds', sc.Variable)
SubframeBounds = NewType('SubframeBounds', sc.Variable)
Choppers = NewType('Choppers', Dict[str, chopper_cascade.Chopper])
SourceChopperName = NewType('SourceChopperName', str)
SourceChopper = NewType('SourceChopper', chopper_cascade.Chopper)
TimeOfFlight = NewType('TimeOfFlight', sc.Variable)
RawSubframeData = NewType('RawSubframeData', sc.DataArray)
UnwrappedData = NewType('UnwrappedData', sc.DataArray)

SourceWavelengthRange = NewType(
    'SourceWavelengthRange', Tuple[sc.Variable, sc.Variable]
)
SourceTimeRange = NewType('SourceTimeRange', Tuple[sc.Variable, sc.Variable])
L1 = NewType('L1', sc.Variable)
L2 = NewType('L2', sc.Variable)

FrameAtSample = NewType('FrameAtSample', chopper_cascade.Frame)


def frame_at_sample(
    source_wavelength_range: SourceWavelengthRange,
    source_time_range: SourceTimeRange,
    choppers: Choppers,
    l1: L1,
) -> FrameAtSample:
    frames = chopper_cascade.FrameSequence.from_source_pulse(
        time_min=source_time_range[0],
        time_max=source_time_range[-1],
        wavelength_min=source_wavelength_range[0],
        wavelength_max=source_wavelength_range[-1],
    )
    frames.chop(choppers.values())
    return frames[-1].propagate_to(l1)


def frame_bounds(frame_at_sample: FrameAtSample, l2: L2) -> FrameBounds:
    bounds = frame_at_sample.bounds()
    return chopper_cascade.propagate_times(**bounds, distance=l2)


def subframe_bounds(frame_at_sample: FrameAtSample, l2: L2) -> SubframeBounds:
    """Used for WFM."""
    bounds = frame_at_sample.subbounds()
    return chopper_cascade.propagate_times(**bounds, distance=l2)


def wrapped_time_offset(da: RawData) -> unwrap.WrappedTimeOffset:
    return da.bins.coords['event_time_offset']


def time_offset(
    wrapped_time_offset: unwrap.WrappedTimeOffset,
    frame_bounds: FrameBounds,
    frame_period: FramePeriod,
) -> unwrap.TimeOffset:
    return unwrap.unwrap(
        wrapped_time_offset=wrapped_time_offset,
        time_offset_min=frame_bounds['bound', 0],
        frame_period=frame_period,
    )


def source_chopper(
    choppers: Choppers, source_chopper_name: SourceChopperName
) -> SourceChopper:
    return choppers[source_chopper_name]


def time_of_flight(
    time_offset: unwrap.TimeOffset,
    source_chopper: SourceChopper,
) -> TimeOfFlight:
    return unwrap.time_of_flight(
        time_offset=time_offset,
        source_time_open=source_chopper.time_open,
        source_time_close=source_chopper.time_close,
    )


def time_of_flight_wfm_old(
    da: RawData,
    time_offset: unwrap.TimeOffset,
    source_chopper: SourceChopper,
    subframe_bounds: SubframeBounds,
) -> TimeOfFlight:
    # RawData -> TimeOffsetData -> TimeOffsetBySubframeData -> TofData
    # Odd that we need data...
    # ...but we first need to compute time_offset, add it to data, bin, get new time_offset
    # Can we add subframe offsets without binning? Using sc.lookup? Or is it too slow?
    # -> See below, actually much faster!
    da = da.copy(deep=False)
    da.bins.coords['time_offset'] = time_offset
    dim = 'time_offset'
    times = subframe_bounds.flatten(dims=['subframe', 'bound'], to=dim)
    da = da.bin({dim: times})[::2]
    time_of_flight = time_of_flight(da.bins.coords[dim], source_chopper)
    return time_of_flight


def time_of_flight_wfm(
    time_offset: unwrap.TimeOffset,
    source_chopper: SourceChopper,
    subframe_bounds: SubframeBounds,
) -> TimeOfFlight:
    # time_zero will have multiple subframes
    times = subframe_bounds.flatten(dims=['subframe', 'bound'], to='subframe')
    neg_shift = sc.zeros(dims=['subframe'], shape=[len(times) - 1], unit='s')
    neg_shift[::2] -= 0.5 * (source_chopper.time_open + source_chopper.time_close)
    lut = sc.DataArray(neg_shift, coords={'subframe': times})
    # Will raise if subframes overlap, since coord for lookup table must be sorted
    out = sc.lookup(lut, dim='subframe')[time_offset]
    out += time_offset
    return out


def tof_data(da: RawData, tof: TimeOfFlight) -> TofData:
    da = da.copy(deep=False)  # todo copy bins
    da.bins.coords['tof'] = tof
    return TofData(da)


def compute_time_of_flight(da: RawData) -> TofData:
    da = da.copy(deep=False)
    time_offset = unwrap.time_offset(da.bins.coords['event_time_offset'])
    return TofData(da)

## Normal

In [ ]:
import sciline as sl

pl = sl.Pipeline(
    [
        wrapped_time_offset,
        time_offset,
        time_of_flight,
        frame_at_sample,
        frame_bounds,
        source_chopper,
        tof_data,
    ]
)
pl[RawData] = sc.DataArray(data=sc.ones(dims=['event_time_offset'], shape=[1000]))
pl[FramePeriod] = sc.scalar(71.0, unit='ms')
pl[SourceTimeRange] = sc.scalar(0.0, unit='ms'), sc.scalar(4.0, unit='ms')
pl[SourceWavelengthRange] = sc.scalar(0.0, unit='angstrom'), sc.scalar(
    10.0, unit='angstrom'
)
pl[Choppers] = {}
pl[SourceChopperName] = 'PSC1'
pl[L1] = sc.scalar(20.0, unit='m')
pl[L2] = sc.scalar(1.0, unit='m')
# pl.get(unwrap.WrappedTimeOffset).visualize()
# pl.get(unwrap.TimeOffset).visualize()
pl.get(TofData).visualize()

## WFM

In [ ]:
import sciline as sl

pl = sl.Pipeline(
    [
        wrapped_time_offset,
        time_offset,
        time_of_flight_wfm,
        frame_at_sample,
        frame_bounds,
        source_chopper,
        tof_data,
        subframe_bounds,
    ]
)
pl[RawData] = sc.DataArray(data=sc.ones(dims=['event_time_offset'], shape=[1000]))
pl[FramePeriod] = sc.scalar(71.0, unit='ms')
pl[SourceTimeRange] = sc.scalar(0.0, unit='ms'), sc.scalar(4.0, unit='ms')
pl[SourceWavelengthRange] = sc.scalar(0.0, unit='angstrom'), sc.scalar(
    10.0, unit='angstrom'
)
pl[Choppers] = {}
pl[SourceChopperName] = 'PSC1'
pl[L1] = sc.scalar(20.0, unit='m')
pl[L2] = sc.scalar(1.0, unit='m')
# pl.get(unwrap.WrappedTimeOffset).visualize()
# pl.get(unwrap.TimeOffset).visualize()
pl.get(TofData).visualize()

In [ ]:
import scipp as sc

nevent = int(1e7)
npix = int(1e6)
table = sc.data.table_xyz(nevent)
da = table.bin(x=npix)
y_bins = sc.array(dims=['y'], values=[0.0, 0.1, 0.2, 0.4, 0.8, 1], unit='m')
hist = sc.DataArray(sc.ones(dims=['y'], shape=[5], unit='m'), coords={'y': y_bins})
# hist = sc.concat([hist]*npix, 'x')
lut = sc.lookup(hist, dim='y')

In [ ]:
%%time
y = da.bins.coords['y']
y -= lut[y]

In [ ]:
%%time
comps = da.bins.constituents
#comps['data'] = comps['data'].copy(deep=False)
comps['data'] = sc.DataArray(comps['data'].data, coords={'y':comps['data'].coords['y']})
tmp = sc.DataArray(sc.bins(**comps))
tmp.bins *= lut
tmp

In [ ]:
%%time
tmp = da.bin(y=y_bins)
tmp.bins.coords['y'] *= hist.data
tmp.bins.concat('y')